# Mineração de Regras de Associação do Dataset SAGE
_Refinamento e busca de padrões para desenvolvimento de experiências gamificadas_

Para o primeiro trabalho da disciplina de Inteligência Computacional, devemos selecionar um dataset público para mineração de dados resultando na coleta de regras de associação.

O dataset utilizado foi publicado através de uma pesquisa com o intuito de ampliar a base de dados ligadas a jogos gamificados com o propósito de analisar as preferências de usuários e como enchergam diversas características ligadas ao game design.

## Link para publicação
[SAGE: A dataset for Smart Adaptive Gamified Education](https://sol.sbc.org.br/index.php/sbie/article/view/26738/26557)

O [dataset original](https://github.com/ArmandoToda/Paper_SBIE2023) pode ser encontrado no repositório do autor no github.

# Metodologia
Criei o repositório e arquivo jupyter para executar as etapas de mineração necessárias para busca de regras de associação:

## Definição de variáveis para operação do projeto

In [15]:

import os
import pandas as pd
import requests

##Variáveis iniciais de referência
csv_filename = "SAGEDataset.csv"
# URL do arquivo XLSX no GitHub
xlsx_url = "https://github.com/ArmandoToda/Paper_SBIE2023/raw/main/DATA.xlsx"
# Nome temporário para o arquivo XLSX baixado
xlsx_filename = "DATA.xlsx"

##Filtro de dados podem ser usados para montar nichos diferentes no dataset
# Exemplo de filtro para o gênero de jogo, país de origem, gênero do jogador, idade, etc.
filterBy = {
    # 'country': ['br', 'us', 'gr', 'uk', 'cn', 'it', 'in', 'es', 'de', 'fr'],
    # 'game_genre': ['action'],
}

min_products = 3
min_support = 0.3
confidence = 0.85


## Obtenção do dataset

In [16]:
if not os.path.exists("assets/"+csv_filename):
    print(f"{csv_filename} não encontrado. Baixando e convertendo...")
    
    # Baixa o arquivo XLSX
    response = requests.get(xlsx_url)
    if response.status_code == 200:
        with open(xlsx_filename, "wb") as file:
            file.write(response.content)
        print(f"Arquivo baixado: {xlsx_filename}")
    else:
        raise Exception(f"Falha ao baixar o arquivo. Status code: {response.status_code}")
    
    """Converte um arquivo XLSX para CSV."""
    try:
        data = pd.read_excel(xlsx_filename)
        data.to_csv("assets/"+csv_filename, index=False)
        print(f"Arquivo convertido para CSV: {csv_filename}")
    except Exception as e:
        raise Exception(f"Erro ao converter XLSX para CSV: {e}")
    
    # Remove o arquivo XLSX temporário
    if os.path.exists(xlsx_filename):
        os.remove(xlsx_filename)
        print(f"Arquivo temporário removido: {xlsx_filename}")
else:
    print(f"{csv_filename} já existe. Nenhuma ação necessária.")

SAGEDataset.csv já existe. Nenhuma ação necessária.


## Preparação dos dados

In [17]:
try:
    df = pd.read_csv("assets/"+csv_filename)  # Use csv_filename em vez de file_path
    
    # Remover duplicatas
    df.drop_duplicates(inplace=True)

    # Remover linhas inválidas (com valores NaN)
    df.dropna(inplace=True)
    
    df['gender'] = df['gender'].apply(lambda x: "male" if x == "Male" else "female" if x == "Female" else "other")
    df['age_group'] = pd.cut(df['age_in_years'], bins=[0, 14, 24, 34, 44, float('inf')], 
                            labels=['<15', '15-24', '25-34', '35-44', '>44'])
    df.drop('age_in_years', axis=1, inplace=True)
    df['years_playing'] = pd.cut(df['years_playing'], bins=[0, 10, 20, 30, float('inf')], 
                                labels=['<10', '10-20', '21-30', '>30'])
    
    # print(df['country'].unique())
    
    country_mapping = {
        'Brazil': 'br',
        'United States': 'us',
        'Greece': 'gr',
        'United Kingdom': 'uk',
        'China': 'cn',
        'Italy': 'it',
        'India': 'in',
        'Spain': 'es',
        'Germany': 'de',
        'France': 'fr',
        'Turkey': 'tr',
        'South Korea': 'kr',
        'Japan': 'jp',
        'Russia': 'ru',
        'Canada': 'ca',
        'Australia': 'au',
        'Mexico': 'mx',
        'Netherlands': 'nl',
        'Argentina': 'ar',
        'Sweden': 'se',
        'Norway': 'no',
        'Finland': 'fi',
        'Denmark': 'dk',
        'Belgium': 'be',
        'Poland': 'pl',
        'Portugal': 'pt',
        'Switzerland': 'ch',
        'Austria': 'at',
        # Adicione outros países conforme necessário
    }
    df['country'] = df['country'].apply(lambda x: country_mapping[x] if x in country_mapping else 'other')
    
    df['time_per_week'] = pd.cut(df['time_per_week'], bins=[0, 5, 15, 25, 30, float('inf')], 
                                labels=['<5', '5-15', '16-25', '26-30', '>30'])
    df['game_genre'] = df['game_genre'].str.split(':').str[0].str.lower()
    
    df['game_setting'] = df['game_setting'].str.contains('Multiplayer', case=False)
    df = df.rename(columns={"game_setting": "setting_multiplayer"})
    
    print("Primeiras linhas do dataset:")
    print(df.head())  # Exibe as primeiras linhas do DataFrame
    
    # Salvar o DataFrame limpo (opcional)
    cleaned_file_path = "assets/Cleaned_" + csv_filename
    df.to_csv(cleaned_file_path, index=False)
    print(f"\nDataset limpo salvo em: {cleaned_file_path}")
except Exception as e:
    print(f"Erro ao carregar o arquivo CSV: {e}")

Primeiras linhas do dataset:
   gender age_group country years_playing time_per_week game_genre  \
0    male     25-34      uk         21-30         26-30     action   
1    male       <15      us           <10         26-30     action   
2  female     15-24      us           <10            <5     action   
3    male     15-24      br         10-20          5-15     action   
4    male     15-24      br         10-20            <5     action   

   setting_multiplayer  points  levels  cooperation  ...  acknowledgment  \
0                False       4       5            3  ...               5   
1                False       2       2            3  ...               1   
2                 True       5       4            4  ...               4   
3                False       5       4            2  ...               3   
4                False       2       3            4  ...               5   

   stats  rarity  imposed_choice  time_pressure  economy  sensation  \
0      4       4      

## Filtro de Canônicos

In [18]:
filtered_df = df.copy()

for column, values in filterBy.items():
        if column in df.columns:
            filtered_df = filtered_df[df[column].isin(values)]
            
# Exibir o DataFrame filtrado
print("\nDataFrame filtrado:")
print(filtered_df.head())  # Exibe as primeiras linhas do DataFrame filtrado


DataFrame filtrado:
   gender age_group country years_playing time_per_week game_genre  \
0    male     25-34      uk         21-30         26-30     action   
1    male       <15      us           <10         26-30     action   
2  female     15-24      us           <10            <5     action   
3    male     15-24      br         10-20          5-15     action   
4    male     15-24      br         10-20            <5     action   

   setting_multiplayer  points  levels  cooperation  ...  acknowledgment  \
0                False       4       5            3  ...               5   
1                False       2       2            3  ...               1   
2                 True       5       4            4  ...               4   
3                False       5       4            2  ...               3   
4                False       2       3            4  ...               5   

   stats  rarity  imposed_choice  time_pressure  economy  sensation  \
0      4       4              

## Binarização de Dataset

In [19]:
from mlxtend.frequent_patterns import apriori, association_rules

columns_to_binarize = ['gender', 'age_group', 'country', 'years_playing', 'time_per_week', 'game_genre', 'setting_multiplayer']
df_binarized = pd.get_dummies(df, columns=columns_to_binarize, prefix=columns_to_binarize)
# Remover itens de columns_to_binarize que estão presentes como chaves em filterBy
columns_to_binarize = [col for col in columns_to_binarize if col not in filterBy.keys()]
# Print se algum item foi removido
if len(columns_to_binarize) != len(df.columns):
    print(f"Removendo {len(columns_to_binarize)} colunas de binarização que estão presentes em filterBy.")

# Exemplo: Transformar a coluna 'points' em colunas binárias 'points_low', 'points_medium', 'points_high'
def transform_scale_to_binary(df, column_name):
    df[f'{column_name}_low'] = df[column_name].apply(lambda x: 1 if x in [1, 2] else 0)
    df[f'{column_name}_medium'] = df[column_name].apply(lambda x: 1 if x == 3 else 0)
    df[f'{column_name}_high'] = df[column_name].apply(lambda x: 1 if x in [4, 5] else 0)
    df.drop(column_name, axis=1, inplace=True)
    
# Aplicar a transformação para as colunas desejadas
columns_to_transform = ['points', 'levels', 'cooperation', 'competition', 'renovation',
                        'progression', 'objectives', 'puzzles', 'novelty', 'chances',
                         'social_pressure', 'acknowledgment', 'stats', 'rarity',
                         'imposed_choice', 'time_pressure', 'economy', 'sensation',
                         'reputation', 'narrative', 'storytelling']  # Substitua pelos nomes das colunas que deseja transformar
for col in columns_to_transform:
    if col in df_binarized.columns:
        transform_scale_to_binary(df_binarized, col)

# Exibir as primeiras linhas do DataFrame binarizado
print("DataFrame binarizado:")
print(df_binarized.head())

# Salvar o DataFrame binarizado (opcional)
binarized_file_path = "assets/Binarized_" + csv_filename
df_binarized.to_csv(binarized_file_path, index=False)
print(f"\nDataset binarizado salvo em: {binarized_file_path}")

Removendo 7 colunas de binarização que estão presentes em filterBy.
DataFrame binarizado:
   gender_female  gender_male  gender_other  age_group_<15  age_group_15-24  \
0          False         True         False          False            False   
1          False         True         False           True            False   
2           True        False         False          False             True   
3          False         True         False          False             True   
4          False         True         False          False             True   

   age_group_25-34  age_group_35-44  age_group_>44  country_at  country_au  \
0             True            False          False       False       False   
1            False            False          False       False       False   
2            False            False          False       False       False   
3            False            False          False       False       False   
4            False            False          

## Execução do Algoritmo Apriori

Nesse processo serão definidos algumas variáveis iniciais e depois a mineração será executada.

In [20]:
frequent_itemsets = apriori(df_binarized, min_support=min_support, use_colnames=True)

print("\nConjuntos frequentes:")
print(frequent_itemsets)

# Gerar regras de associação
num_itemsets = df.columns.size # Número total de colunas no começo do tratamento
rules = association_rules(frequent_itemsets, num_itemsets=num_itemsets, metric="confidence", min_threshold=confidence)

#Aplicando filtro no antecedents e consequents dos resultados
rules = rules[rules['antecedents'].apply(len) + rules['consequents'].apply(len) > min_products]


/Users/realima/Work/UFPA/PythonProjects/SAGEMining/.venv/lib/python3.12/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type




Conjuntos frequentes:
      support                                           itemsets
0    0.320042                                    (gender_female)
1    0.672613                                      (gender_male)
2    0.375656                                  (age_group_15-24)
3    0.398216                                  (age_group_25-34)
4    0.458027                                       (country_us)
..        ...                                                ...
815  0.315845  (narrative_high, setting_multiplayer_False, pr...
816  0.306925  (stats_high, narrative_high, progression_high,...
817  0.317943  (narrative_high, progression_high, objectives_...
818  0.327912  (narrative_high, progression_high, objectives_...
819  0.317943  (stats_high, narrative_high, progression_high,...

[820 rows x 2 columns]


# Plotting de Resultados
Pesquisando maneiras de visualizar os resultados de associação encontrei um artigo que apresentava de maneira intuitiva e interativa o plot do 

In [21]:
import plotly.express as px
import webbrowser

# Converter 'antecedents' e 'consequents' de frozenset para string
rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

fig = px.scatter_3d(
    rules,
    x='confidence',  # Eixo X
    y='lift',        # Eixo Y
    z='certainty',   # Eixo Z
    color='lift',    # Cor baseada no lift
    size='support',  # Tamanho dos pontos baseado no suporte
    hover_data=['antecedents', 'consequents'],  # Mostrar os antecedentes e consequentes ao passar o mouse
    title="Regras de Associação: Confidence, Lift e Certainty"
)

# Exibir o gráfico
fig.write_html('plot.html')
# Caminho para o arquivo HTML gerado
plot_file_path = os.path.abspath("plot.html")

# Abrir o arquivo no navegador padrão
webbrowser.open(f"file://{plot_file_path}")
# Exibir as regras de associação
print("\nRegras de associação:")
rule_file_path = "assets/rules_" + csv_filename
rules.to_csv(rule_file_path, index=False)
print(rules) #Também é possível acessar depois da execução o wrangler das regras


Regras de associação:
                                           antecedents  \
283  progression_high, setting_multiplayer_False, g...   
284  objectives_high, setting_multiplayer_False, ge...   
285  sensation_high, setting_multiplayer_False, gen...   
286  narrative_high, setting_multiplayer_False, gen...   
287  storytelling_high, setting_multiplayer_False, ...   
..                                                 ...   
867  stats_high, narrative_high, objectives_high, s...   
868  stats_high, progression_high, objectives_high,...   
869  objectives_high, stats_high, narrative_high, s...   
870  sensation_high, stats_high, narrative_high, st...   
871  sensation_high, stats_high, objectives_high, s...   

                             consequents  antecedent support  \
283                      objectives_high            0.401364   
284                     progression_high            0.375656   
285                     progression_high            0.343652   
286                     